<a href="https://colab.research.google.com/github/JamesKha/MovieRecommendation/blob/main/CollabFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
linksDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/links.csv')
moviesDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/movies.csv')
ratingsDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/ratings.csv')
tagsDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/tags.csv')

In [3]:
movieAndRatings = pd.merge(moviesDF,ratingsDF, on='movieId' )

In [4]:
movieAndRatings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [5]:
movieAndRatings.groupby('userId')['title'].apply(lambda x: list(np.unique(x)))

userId
1      [13th Warrior, The (1999), 20 Dates (1998), Ab...
2      [Collateral (2004), Dark Knight Rises, The (20...
3      [2012 (2009), Alien Contamination (1980), Andr...
4      [12 Angry Men (1957), Adventures of Priscilla,...
5      [Ace Ventura: Pet Detective (1994), Addams Fam...
                             ...                        
606    [11'09"01 - September 11 (2002), 2001: A Space...
607    [Adventures of Milo and Otis, The (Koneko mono...
608    [10 Items or Less (2006), 187 (One Eight Seven...
609    [Adventures of Pinocchio, The (1996), Anne Fra...
610    ['71 (2014), (500) Days of Summer (2009), 10 C...
Name: title, Length: 610, dtype: object

In [6]:
movieAndRatings.title.value_counts()

Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
                                   ... 
Jimmy Carr: Telling Jokes (2009)      1
Survive Style 5+ (2004)               1
Mortuary (1983)                       1
The Dark Tower (2017)                 1
Burlesque (2010)                      1
Name: title, Length: 9719, dtype: int64

In [7]:
pd.DataFrame(movieAndRatings.groupby(['userId','genres'])['rating'].max()).to_csv('/content/drive/MyDrive/testfile.csv')

In [8]:
user_movie_rating = movieAndRatings.pivot_table(index='userId', columns='title', values='rating')

In [9]:
user_movie_rating

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...All the Marbles (1981),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),1-900 (06) (1994),10 (1979),10 Cent Pistol (2015),10 Cloverfield Lane (2016),10 Items or Less (2006),10 Things I Hate About You (1999),10 Years (2011),"10,000 BC (2008)",100 Girls (2000),100 Streets (2016),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),101 Dalmatians II: Patch's London Adventure (2003),101 Reykjavik (101 Reykjavík) (2000),102 Dalmatians (2000),10th & Wolf (2006),"10th Kingdom, The (2000)","10th Victim, The (La decima vittima) (1965)","11'09""01 - September 11 (2002)",11:14 (2003),"11th Hour, The (2007)",12 Angry Men (1957),12 Angry Men (1997),12 Chairs (1971),12 Chairs (1976),12 Rounds (2009),12 Years a Slave (2013),...,Zathura (2005),Zatoichi and the Chest of Gold (Zatôichi senryô-kubi) (Zatôichi 6) (1964),Zazie dans le métro (1960),Zebraman (2004),"Zed & Two Noughts, A (1985)",Zeitgeist: Addendum (2008),Zeitgeist: Moving Forward (2011),Zeitgeist: The Movie (2007),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zero Effect (1998),"Zero Theorem, The (2013)",Zero de conduite (Zero for Conduct) (Zéro de conduite: Jeunes diables au collège) (1933),Zeus and Roxanne (1997),Zipper (2015),Zodiac (2007),Zombeavers (2014),Zombie (a.k.a. Zombie 2: The Dead Are Among Us) (Zombi 2) (1979),Zombie Strippers! (2008),Zombieland (2009),Zone 39 (1997),"Zone, The (La Zona) (2007)",Zookeeper (2011),Zoolander (2001),Zoolander 2 (2016),Zoom (2006),Zoom (2015),Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [10]:
test_ratings = user_movie_rating["Shawshank Redemption, The (1994)"]

In [11]:
movies_like_test = user_movie_rating.corrwith(test_ratings)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [12]:
corr_test = pd.DataFrame(movies_like_test, columns=['Correlation'])
corr_test.dropna(inplace=True)
corr_test.head()

,Correlation
title,
"'burbs, The (1989)",0.419543
(500) Days of Summer (2009),0.249580
*batteries not included (1987),0.404520
...And Justice for All (1979),-1.000000
10 Cloverfield Lane (2016),0.145671


In [13]:
corr_test.sort_values('Correlation', ascending=False).head(10)

,Correlation
title,
"Principal, The (1987)",1.0
Chi-Raq (2015),1.0
Rififi (Du rififi chez les hommes) (1955),1.0
Robot & Frank (2012),1.0
Rock Star (2001),1.0
Catch a Fire (2006),1.0
"Cat in the Hat, The (2003)",1.0
Cars 2 (2011),1.0
Runaway (1984),1.0


In [14]:
newDataFrameTest = pd.DataFrame(movieAndRatings.groupby('title')['rating']\
.agg(['count','mean'])\
.sort_values(['count', 'mean'], ascending=[False, True]))

In [15]:
newDataFrameTest = newDataFrameTest.rename({'count':'vote_count',\
                         'mean':'vote_average'}, axis='columns')
newDataFrameTest

,vote_count,vote_average
title,,
Forrest Gump (1994),329,4.164134
"Shawshank Redemption, The (1994)",317,4.429022
Pulp Fiction (1994),307,4.197068
"Silence of the Lambs, The (1991)",279,4.161290
"Matrix, The (1999)",278,4.192446
...,...,...
Won't You Be My Neighbor? (2018),1,5.000000
Wonder Woman (2009),1,5.000000
World of Glory (1991),1,5.000000


###There is an uneven amount of votes/ratings per movie, so the average of the movie would be used to correct them accordingly

In [16]:
quantile_ninety = newDataFrameTest['vote_count'].quantile(0.90)
mean = newDataFrameTest['vote_average'].mean()
def weighted_rating(x, quantile_ninety=quantile_ninety, mean=mean):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+quantile_ninety) * R) + (quantile_ninety/(quantile_ninety+v) * mean)
newDataFrameTest['score'] = newDataFrameTest.apply(weighted_rating, axis=1)
newDataFrameTest

,vote_count,vote_average,score
title,,,
Forrest Gump (1994),329,4.164134,4.095743
"Shawshank Redemption, The (1994)",317,4.429022,4.337455
Pulp Fiction (1994),307,4.197068,4.121510
"Silence of the Lambs, The (1991)",279,4.161290,4.081975
"Matrix, The (1999)",278,4.192446,4.110113
...,...,...,...
Won't You Be My Neighbor? (2018),1,5.000000,3.324446
Wonder Woman (2009),1,5.000000,3.324446
World of Glory (1991),1,5.000000,3.324446


In [17]:
ratingsDF

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [18]:
from collections import defaultdict

def top_rated_movies(predictions, numberOfPredictions=10):

    top_predictions = defaultdict(list)
    for user_id, iid, true_r, est, _ in predictions:
        top_predictions[user_id].append((iid, est))

    for user_id, user_ratings in top_predictions.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_predictions[user_id] = user_ratings[:numberOfPredictions]

    return top_predictions


In [20]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 51 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619403 sha256=68c3e5199d7082d56b7788fe4d99bdbf86e604dd614e983c12c1e8639e697a9b
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [28]:
from surprise import SVD, accuracy,KNNBasic,KNNWithMeans,KNNWithZScore,KNNBaseline,SVDpp
from surprise import Dataset as Dataset
from surprise import Reader as Reader
from surprise.model_selection import train_test_split,GridSearchCV

In [29]:
reader = Reader(rating_scale=(1, 5))
df = Dataset.load_from_df(ratingsDF[['userId', 	'movieId', 	'rating' 	]], reader=reader)

In [30]:
raw_ratings = df.raw_ratings

In [31]:
threshold = int(.7 * len(raw_ratings))                                     
trainset_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings = raw_ratings[threshold:]        

In [32]:
df.raw_ratings = trainset_raw_ratings

In [33]:
param_grid = {
    'n_factors':[5, 10,20],
    'n_epochs': [5, 10, 20], 
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(df)

In [34]:
algo = gs.best_estimator['rmse']

In [35]:
trainset = df.build_full_trainset()
algo.fit(trainset)

In [37]:
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)


Biased accuracy on A,   RMSE: 0.8530


0.8529636442828266

In [38]:
top_n = top_rated_movies(predictions,numberOfPredictions=10)

In [39]:
for user_id, user_ratings in top_n.items():
    print(user_id, [iid for (iid, _) in user_ratings])

1 [50, 296, 2959, 1208, 1213, 527, 2329, 1196, 1197, 608]
2 [318, 58559, 68157, 48516, 112552, 1704, 6874, 74458, 79132, 91529]
3 [720, 527, 1272, 3703, 3949, 6835, 5919, 5746, 914, 5181]
4 [1250, 898, 3508, 912, 296, 2959, 475, 1213, 904, 2019]
5 [318, 50, 296, 475, 527, 608, 290, 58, 110, 364]
6 [318, 50, 296, 475, 527, 608, 593, 356, 1084, 293]
7 [750, 50, 1208, 5618, 2019, 1196, 48516, 260, 1210, 593]
8 [318, 50, 296, 527, 593, 356, 32, 110, 47, 364]
9 [922, 2300, 1198, 4993, 923, 5952, 3328, 5902, 223, 1270]
10 [912, 296, 2959, 58559, 2571, 356, 7153, 49272, 4993, 5952]
11 [318, 1210, 593, 356, 1704, 2028, 6, 110, 457, 529]
12 [1265, 1357, 838, 6942, 920, 3668, 261, 8533, 3967, 2572]
13 [1198, 2571, 4011, 47, 3578, 3897, 3996, 1639, 3793, 3300]
14 [318, 296, 527, 593, 356, 553, 32, 110, 47, 150]
15 [318, 296, 2959, 858, 5618, 58559, 527, 2329, 1196, 260]
16 [318, 750, 1261, 912, 50, 296, 2959, 1193, 858, 913]
17 [318, 750, 3508, 50, 296, 2959, 1193, 858, 5618, 1213]
18 [318, 17759